In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd

In [18]:
df = pd.read_excel("run-created.xlsx")

In [19]:
df.head()

,Player,At Bats,Hits,1B,2B,3B,HR,GDP,SH,SF,SB,CS,Estimated Outs,Other Outs,BB+HBP,Runs Created,Game Outs Used,Runs Created / Game
0,Bryant (2016),603,176,99,35,3,39,3,0,3,8,5,416.146,11,93,129.089080,427.146,8.108375
1,Trout (2016),549,173,107,32,5,29,5,0,5,30,7,366.118,17,127,134.023669,383.118,9.385764
2,Cabrera (2013),555,193,122,26,1,44,19,0,2,3,0,352.010,21,95,156.406154,373.010,11.250039


In [20]:
####
#
# Constants based on actuals computes for total outs 
# per game.
#
####
df['computed_outs_per_game'] = 26.72
df['season_total_outs'] = 26.72 * 162

In [21]:
df['scale_factor'] = df.apply(lambda row: row['season_total_outs']/row['Estimated Outs'], axis=1)

In [22]:
    def predicted_run(row):
        return -411.81 + (0.451 * row['1B']) + (0.805 * row['2B']) + (1.072 * row['3B']) + (1.428 * row['HR']) + (0.326 * row['BB+HBP']) + (0.250 * row['SB']) # - (0.254 * [row['CS']])

In [29]:
def scaled_predicted_run(row):
    return -411.81 + (0.451 * row['scaled_1B']) + (0.805 * row['scaled_2B']) + (1.072 * row['scaled_3B']) + (1.428 * row['scaled_HR']) + (0.326 * row['scaled_BB+HBP']) + (0.250 * row['scaled_SB']) # - (0.254 * [row['CS']])

In [25]:
####
#
# Scaled metrics to be used with linear model coefficients
#
###
df['scaled_AB']=df.apply(lambda row: row['scale_factor'] * row['At Bats'], axis=1)
df['scaled_Hits']=df.apply(lambda row: row['scale_factor'] * row['Hits'], axis=1)
df['scaled_1B']=df.apply(lambda row: row['scale_factor'] * row['1B'], axis=1)
df['scaled_2B']=df.apply(lambda row: row['scale_factor'] * row['2B'], axis=1)
df['scaled_3B']=df.apply(lambda row: row['scale_factor'] * row['3B'], axis=1)
df['scaled_HR']=df.apply(lambda row: row['scale_factor'] * row['HR'], axis=1)
df['scaled_BB+HBP']=df.apply(lambda row: row['scale_factor'] * row['BB+HBP'], axis=1)
df['scaled_SB']=df.apply(lambda row: row['scale_factor'] * row['SB'], axis=1)

In [26]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [27]:
df.head()

,Player,At Bats,Hits,1B,2B,3B,HR,GDP,SH,SF,SB,CS,Estimated Outs,Other Outs,BB+HBP,Runs Created,Game Outs Used,Runs Created / Game,computed_outs_per_game,season_total_outs,scale_factor,scaled_AB,scaled_Hits,scaled_1B,scaled_2B,scaled_3B,scaled_HR,scaled_BB+HBP,scaled_SB
0,Bryant (2016),603,176,99,35,3,39,3,0,3,8,5,416.146,11,93,129.089080,427.146,8.108375,26.72,4328.64,10.401734,6272.245606,1830.705185,1029.771667,364.060690,31.205202,405.667626,967.361263,83.213872
1,Trout (2016),549,173,107,32,5,29,5,0,5,30,7,366.118,17,127,134.023669,383.118,9.385764,26.72,4328.64,11.823073,6490.867316,2045.391704,1265.068858,378.338350,59.115367,342.869130,1501.530326,354.692203
2,Cabrera (2013),555,193,122,26,1,44,19,0,2,3,0,352.010,21,95,156.406154,373.010,11.250039,26.72,4328.64,12.296923,6824.792477,2373.306213,1500.224653,319.720008,12.296923,541.064629,1168.207721,36.890770


In [30]:
df['linear_weight_runs'] = df.apply(scaled_predicted_run, axis=1)

In [31]:
df.head()

,Player,At Bats,Hits,1B,2B,3B,HR,GDP,SH,SF,SB,CS,Estimated Outs,Other Outs,BB+HBP,Runs Created,Game Outs Used,Runs Created / Game,computed_outs_per_game,season_total_outs,scale_factor,scaled_AB,scaled_Hits,scaled_1B,scaled_2B,scaled_3B,scaled_HR,scaled_BB+HBP,scaled_SB,linear_weight_runs
0,Bryant (2016),603,176,99,35,3,39,3,0,3,8,5,416.146,11,93,129.089080,427.146,8.108375,26.72,4328.64,10.401734,6272.245606,1830.705185,1029.771667,364.060690,31.205202,405.667626,967.361263,83.213872,1294.594464
1,Trout (2016),549,173,107,32,5,29,5,0,5,30,7,366.118,17,127,134.023669,383.118,9.385764,26.72,4328.64,11.823073,6490.867316,2045.391704,1265.068858,378.338350,59.115367,342.869130,1501.530326,354.692203,1594.459154
2,Cabrera (2013),555,193,122,26,1,44,19,0,2,3,0,352.010,21,95,156.406154,373.010,11.250039,26.72,4328.64,12.296923,6824.792477,2373.306213,1500.224653,319.720008,12.296923,541.064629,1168.207721,36.890770,1698.046926


In [32]:
df['runs_per_game'] = df.apply(lambda row: row['linear_weight_runs']/162, axis=1)

In [33]:
df.head()

,Player,At Bats,Hits,1B,2B,3B,HR,GDP,SH,SF,SB,CS,Estimated Outs,Other Outs,BB+HBP,Runs Created,Game Outs Used,Runs Created / Game,computed_outs_per_game,season_total_outs,scale_factor,scaled_AB,scaled_Hits,scaled_1B,scaled_2B,scaled_3B,scaled_HR,scaled_BB+HBP,scaled_SB,linear_weight_runs,runs_per_game
0,Bryant (2016),603,176,99,35,3,39,3,0,3,8,5,416.146,11,93,129.089080,427.146,8.108375,26.72,4328.64,10.401734,6272.245606,1830.705185,1029.771667,364.060690,31.205202,405.667626,967.361263,83.213872,1294.594464,7.991324
1,Trout (2016),549,173,107,32,5,29,5,0,5,30,7,366.118,17,127,134.023669,383.118,9.385764,26.72,4328.64,11.823073,6490.867316,2045.391704,1265.068858,378.338350,59.115367,342.869130,1501.530326,354.692203,1594.459154,9.842340
2,Cabrera (2013),555,193,122,26,1,44,19,0,2,3,0,352.010,21,95,156.406154,373.010,11.250039,26.72,4328.64,12.296923,6824.792477,2373.306213,1500.224653,319.720008,12.296923,541.064629,1168.207721,36.890770,1698.046926,10.481771
